In [6]:
%matplotlib inline
import matplotlib
import numpy as np
import pandas as pd
from numpy import genfromtxt
import scipy.misc
import matplotlib.pyplot as plt
import sklearn.datasets
import theano
import theano.tensor as T
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten, Convolution2D, MaxPooling2D
from keras.layers import advanced_activations
from keras.layers.normalization import BatchNormalization
from keras.optimizers import *
from keras.layers.advanced_activations import *
from keras.regularizers import l2
import csv
import pickle
from pandas.io.parsers import read_csv
from sklearn.utils import shuffle
from sklearn.externals import joblib

In [7]:
IMAGE_WIDTH = 96
IMAGE_HIGHT = 96
CHANNELS_NUM = 1
DATASET_SIZE = 7049
CLASSES_NUM = 15*2
EPOCHS_NUM = 500
BATCH_SIZE = 16
LEARNING_RATE = 0.001
FTRAIN = 'CSVs/facial_points_training.csv'
FTEST = 'CSVs/facial_points_test.csv'
FLOOKUP = 'CSVs/facial_points_IdLookupTable.csv'

In [8]:
def plot_sample(x, y, axis):
    img = x.reshape(96, 96)
    axis.imshow(img, cmap='gray')
    if y is not None:
        axis.scatter(y[0::2] * 48 + 48, y[1::2] * 48 + 48, marker='x', s=10)
        
def make_submission(test_labels):
    test_labels *= 48.0
    test_labels += 48.0
    test_labels = test_labels.clip(0, 96)

    lookup_table = pd.read_csv(FLOOKUP)
    values = []

    cols = joblib.load('cols.pkl')

    for index, row in lookup_table.iterrows():
        values.append((
            row['RowId'],
            test_labels[row.ImageId - 1][np.where(cols == row.FeatureName)[0][0]],
            ))
    submission = pd.DataFrame(values, columns=('RowId', 'Location'))
    submission.to_csv('CSVs/submission.csv', index=False)
    
def load(test=False, cols=None):
    fname = FTEST if test else FTRAIN
    df = read_csv(fname)  
    cols_l = df.columns[:-1]
    df['Image'] = df['Image'].apply(lambda im: np.fromstring(im, sep=' '))
    if cols:  
        df = df[list(cols) + ['Image']]

    print(df.count())  
    df = df.dropna()
    X = np.vstack(df['Image'].values)  
    X = X.astype(np.float32)

    if not test:  # only FTRAIN has any target columns
        y = df[df.columns[:-1]].values
        y = (y - 48) / 48  # scale target coordinates to [-1, 1]
        X, y = shuffle(X, y, random_state=42)  # shuffle train data
        y = y.astype(np.float32)
        joblib.dump(cols_l, 'cols.pkl', compress=3)
    else:
        y = None
    return X, y

In [5]:
X, y = load()
print("X.shape == {}; X.min == {:.3f}; X.max == {:.3f}".format(X.shape, X.min(), X.max()))
print("y.shape == {}; y.min == {:.3f}; y.max == {:.3f}".format(y.shape, y.min(), y.max()))

left_eye_center_x            7039
left_eye_center_y            7039
right_eye_center_x           7036
right_eye_center_y           7036
left_eye_inner_corner_x      2271
left_eye_inner_corner_y      2271
left_eye_outer_corner_x      2267
left_eye_outer_corner_y      2267
right_eye_inner_corner_x     2268
right_eye_inner_corner_y     2268
right_eye_outer_corner_x     2268
right_eye_outer_corner_y     2268
left_eyebrow_inner_end_x     2270
left_eyebrow_inner_end_y     2270
left_eyebrow_outer_end_x     2225
left_eyebrow_outer_end_y     2225
right_eyebrow_inner_end_x    2270
right_eyebrow_inner_end_y    2270
right_eyebrow_outer_end_x    2236
right_eyebrow_outer_end_y    2236
nose_tip_x                   7049
nose_tip_y                   7049
mouth_left_corner_x          2269
mouth_left_corner_y          2269
mouth_right_corner_x         2270
mouth_right_corner_y         2270
mouth_center_top_lip_x       2275
mouth_center_top_lip_y       2275
mouth_center_bottom_lip_x    7016
mouth_center_b

In [13]:
def build_model():
    model = Sequential()
    
    model.add(Convolution2D(16, 3, 3, init = 'he_normal', W_regularizer = l2(1e-4), border_mode='same',
                            input_shape=(IMAGE_HIGHT, IMAGE_WIDTH, CHANNELS_NUM)))
    model.add(normalization.BatchNormalization())
    model.add(LeakyReLU(alpha=0.15))
    model.add(Dropout(0.4))
    
    model.add(Convolution2D(32, 3, 3, init = 'he_normal', W_regularizer = l2(1e-4), border_mode='same'))
    model.add(normalization.BatchNormalization())
    model.add(LeakyReLU(alpha=0.15))
    model.add(Dropout(0.4))
    
    model.add(Convolution2D(16, 3, 3, init = 'he_normal', W_regularizer = l2(1e-4), border_mode='same'))
    model.add(normalization.BatchNormalization())
    model.add(LeakyReLU(alpha=0.15))
    model.add(Dropout(0.4))
    
    model.add(Convolution2D(8, 1, 1, init = 'he_normal', W_regularizer = l2(1e-4), border_mode='same'))
    model.add(normalization.BatchNormalization())
    model.add(LeakyReLU(alpha=0.15))
    model.add(Dropout(0.4))

    model.add(Flatten())
    model.add(Dense(64, init = 'he_normal', W_regularizer = l2(1e-4)))
    model.add(normalization.BatchNormalization())
    model.add(LeakyReLU(alpha=0.15))
    model.add(Dropout(0.4))
    
    model.add(Dense(32, init = 'he_normal', W_regularizer = l2(1e-4)))
    model.add(normalization.BatchNormalization())
    model.add(LeakyReLU(alpha=0.15))
    model.add(Dropout(0.4))

    model.add(Dense(CLASSES_NUM, init = 'he_normal', W_regularizer = l2(1e-4)))
    model.add(Activation('linear'))
    return model

In [14]:
inzeption = build_model()
inzeption.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_9 (Convolution2D)  (None, 96, 96, 16)    160         convolution2d_input_3[0][0]      
____________________________________________________________________________________________________
batchnormalization_11 (BatchNorm (None, 96, 96, 16)    32          convolution2d_9[0][0]            
____________________________________________________________________________________________________
leakyrelu_11 (LeakyReLU)         (None, 96, 96, 16)    0           batchnormalization_11[0][0]      
____________________________________________________________________________________________________
dropout_11 (Dropout)             (None, 96, 96, 16)    0           leakyrelu_11[0][0]               
___________________________________________________________________________________________

In [ ]:
adam = Adam(lr=LEARNING_RATE, decay=0.001)
inzeption.compile(optimizer=adam, loss='mse', metrics=['accuracy'])
try:
    inzeption.fit(X.reshape(-1, 96, 96, 1), y, batch_size=32, nb_epoch=100, validation_split= 0.1, shuffle=True)
except KeyboardInterrupt:
    pass

Train on 1926 samples, validate on 214 samples
Epoch 1/100
 160/1926 [=>............................] - ETA: 244s - loss: 1.5942 - acc: 0.0187

In [ ]:
inzeption.save('my_2nd_model.h5')

In [ ]:
sample1 = load(test=True)[0][19:20].reshape(1, 96, 96, 1)
y_pred1 = inzeption.predict(sample1)[0]
print y_pred1.shape
fig = plt.figure(figsize=(9, 9))
ax = fig.add_subplot(1, 1, 1, xticks=[], yticks=[])
plot_sample(sample1, y_pred1, ax)
plt.show()

In [ ]:
tst = load(test=True)[0].reshape(-1, 96, 96, 1)
y_pred1 = inzeption.predict(tst)
print y_pred1.shape, tst.shape
make_submission(y_pred1)